<a href="https://colab.research.google.com/github/NataliaRusinchuk/Process_surveys/blob/main/Process_results_for_Program.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
#@title <-- Press the button to start the processing and everything will be done automatically { display-mode: "form" }
!pip install python-docx 
import sys
import os
from google.colab import output
from google.colab import files
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from docx import Document
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.shared import Cm
from docx.shared import RGBColor
from docx.shared import Pt
from docx2pdf import convert

# Labels used for evaluation
LABELS = {'5 Задоволений': 5, '4 Скоріше задоволений': 4, '3 Важко відповісти': 3, '2 Скоріше не задоволений': 2, '1 Не задоволений': 1}

# Functions used in classes
# Labeling the bars
def autolabel(rects, b):
    #Attach a text label above each bar in *rects*, displaying its height
    for rect in rects:
        height = rect.get_height()
        b.annotate('{}'.format(height),
                    xy = (rect.get_x() + rect.get_width() / 2, height),
                    xytext = (0, 2),  # 3 points vertical offset
                    textcoords = "offset points",
                    ha = 'center', va = 'bottom',
                    fontsize = 20)

# Adding line breaks to titles
def process_title(string):
    a = string.split()
    if len(a) > 5:
        string = ' '.join([elem for elem in a[:5]])
        string += '\n'
        string += ' '.join([elem for elem in a[5:]])
    return string

# Classes
class Question_with_marks:
    def __init__(self, data, j):
        # j is number of question
        self.people_total = len(data.index)
        self.name = process_title(data.columns[j])
        self.j = j
        
        # Class for Answers for some question 
        class Answer:
            def __init__(self, data, i, j):
                # i is number of answer
                # j is number of question
                self.mark = data.iloc[i,j]
                date, year = (data.iloc[i,0], data.iloc[i,1])  
                self.semester = int(year[0]) + 2022 - date.year

        def Collect_answers(data, j):
            # j is number of question 
            n = len(data.index)
            answers = []
            for i in range(n):
                answers += [Answer(data, i, j)]    
            return answers
        
        self.answers = Collect_answers(data, j)
        self.semesters_total = max([ans.semester for ans in self.answers])

    def find_marks(self):
        marks = np.zeros(self.semesters_total)
        people = np.zeros(self.semesters_total)
        for i in range(self.people_total):
            temp = np.zeros(self.semesters_total)
            temp[self.answers[i].semester - 1] = 1
            people += temp
            marks += temp * self.answers[i].mark
        marks = np.round(
            [marks[i] / people[i] if people[i] != 0 
            else 0 
            for i in range(len(marks))]
        )
        return marks, people  
        
    # Create figure, plot results, save them as *.png, close the figure and return ist adress
    def plot_results(self):
        marks, people = self.find_marks()
            # Define an average mark for answers in June (now)
        mark_now = round(
            np.sum(marks[1::2] * people[1::2])
            / np.sum(people[1::2]), 2
        )
        # Define an average mark for answers in December (was semester ago)
        mark_was = round(
            np.sum(marks[::2] * people[1::2])
            / np.sum(people[::2]), 2
        )
        fig, [a, b] = plt.subplots(nrows=1, ncols=2, figsize=(18,9))
        # Title of all the figure
        plt.subplots_adjust(top = 0.75)
        fig.suptitle(self.name, 
                     y = 1, 
                     verticalalignment='top',
                     weight = 'bold',
                     fontsize = 28, 
                     linespacing = 1.5)
        # Bars plot for marks for different semesters
        semesters = [str(x) for x in range(1, self.semesters_total + 1)]
        rects1 = a.bar(semesters, marks, color='b', align='center', edgecolor='black', width=0.5)
        a.set_title('Результати опитування за семестрами навчання', size = 20)
        a.set_ylabel('Рівень задоволення',size=22)
        a.set_xlabel('Семестр', size=22)
        a.grid(True)
        a.set_ylim(0, 5.5)
        a.set_xticklabels(semesters, size=20)
        a.set_yticklabels(range(6), size=20)
        autolabel(rects1, a)
        b.text(0.5, 0.85, 'Поточний рівень:', 
               fontsize = 30, 
               horizontalalignment = 'center', 
               verticalalignment='bottom')
        b.text(0.5, 0.55, mark_now, 
               fontsize = 90,
               color = 'red',
               weight = 'bold',
               horizontalalignment = 'center', 
               verticalalignment='bottom')   
        b.text(0.5, 0.45, f'Опитано студентів: {round(np.sum(people[1::2]))}', 
               fontsize = 30,
               horizontalalignment = 'center', 
               verticalalignment='top')
        b.text(0.5, 0.25, f'Результат попереднього семестру:\n'
               + f'{mark_was}, опитано {round(np.sum(people[::2]))} студентів', 
               fontsize = 20,
               horizontalalignment = 'center', 
               verticalalignment='top')
        b.set_xticks([])
        b.set_yticks([])
        figadress = f'{program}, питання {self.j - 1}.png'
        fig.savefig(figadress)
        plt.close()
        return figadress
    
    def add_to_doc(self, document):
        pict = self.plot_results()
        p = document.add_paragraph()
        p.alignment = WD_ALIGN_PARAGRAPH.CENTER
        run = p.add_run()
        run.add_picture(pict, width=Cm(15))
        os.remove(pict)
    
class Question_without_marks(Question_with_marks):
    
        def find_marks(self):
            dict = {}
            for i in range(self.people_total):
                if self.answers[i].mark not in dict:
                    dict[self.answers[i].mark] = 0
                dict[self.answers[i].mark] += 1
            if 'Ще не обирали' in dict: del dict['Ще не обирали']
            return dict
         
        def plot_results(self):    
            marks = self.find_marks()
            fig, a = plt.subplots(nrows=1, ncols=1, figsize=(9,9))
            plt.subplots_adjust(top = 0.75)
            fig.suptitle(self.name, 
                         y = 1, 
                         verticalalignment='top',
                         weight = 'bold',
                         fontsize = 28, 
                         linespacing = 1.5)
            # Data to plot
            labels = []
            sizes = []
            for x, y in marks.items():
                labels.append(x)
                sizes.append(y)
            def absolute_value(val):
                a  = np.round(val/100.*np.asarray(sizes).sum(), 0)
                return a
            colors = ('green', 'gold', 'red')
            a.pie(sizes, labels=None, colors=colors, autopct=absolute_value, textprops={'fontsize': 24})
            a.legend(labels = labels, fontsize=20)
            figadress = f'{program}, питання {self.j - 1}.png'
            fig.savefig(figadress)
            plt.close()
            return figadress

        def add_to_doc(self, paragraph):
            pict = self.plot_results()
            run = paragraph.add_run()
            run.add_picture(pict, width=Cm(6))
            os.remove(pict)
    
def add_program_title(program, document):
    p = document.add_paragraph()
    p.alignment = WD_ALIGN_PARAGRAPH.CENTER
    run = p.add_run(f'{program}\n\n')
    run.font.color.rgb = RGBColor(0x42, 0x24, 0xE9)
    run.font.size = Pt(30)
    run.bold = True
    
def process_program(program):
    uploaded = files.upload()
    for fn in uploaded.keys():
      xl = pd.ExcelFile(fn)
      data = xl.parse()
    for i in range(len(data.iloc[:,3])):
        data.iloc[i,3] = data.iloc[i,3][:13]
        for j in range(5, len(data.columns) - 1):
            data.iloc[i,j] = int(data.iloc[i,j][0])
    document = Document()
    add_program_title(program, document)
    Question_with_marks(data, 2).add_to_doc(document)
    p = document.add_paragraph()
    p.alignment = WD_ALIGN_PARAGRAPH.CENTER
    Question_without_marks(data, 3).add_to_doc(p)
    Question_without_marks(data, 4).add_to_doc(p)
    for i in range(5, 20):
        Question_with_marks(data, i).add_to_doc(document)
    document.add_paragraph('Відгуки студентів:')
    for i in range(len(data.index)):
        if type(data.iloc[i, -1]) == str:
            document.add_paragraph(data.iloc[i, -1])
    document.save(f'Report for {program} {YEAR}.docx')
    files.download(f'Report for {program} {YEAR}.docx') 

output.clear()
# The survey is conducted twice during the academic year: in December and in June
# Here is an example for 2021-2022 academic year
# End year of the survey
print('Let\'s begin!\n')
YEAR = input ('The survey is conducted twice during the academic year: in December and in June\n' 
                + 'Here is an example for 2021-2022 academic year\n'
                + 'End year of the example survey is 2022\n\n'
                + 'Please, enter the end year of your survey: ') 
program = input('\nEnter the name of the Program to save the report with this name: ')
print('\nChoose the file with the survey results in *.xlsx')
process_program(program)
print('\n Done!')

Let's begin!

The survey is conducted twice during the academic year: in December and in June
Here is an example for 2021-2022 academic year
End year of the example survey is 2022

Please, enter the end year of your survey: 2022

Enter the name of the Program to save the report with this name: Program 1

Choose the file with the survey results in *.xlsx


Saving Results for Program.xlsx to Results for Program (2).xlsx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


 Done!
